In [1]:
from epo.tipdata.patstat import PatstatClient
patstat = PatstatClient()

This client instance is currently configured to use a test dataset with reduced number of publications (~10K).
Use PatstatClient(env='PROD') to use the complete PATSTAT dataset (>140M publications).
Use PatstatClient(env='TEST') to use the test dataset and avoid displaying this warning



In [ ]:
from epo.tipdata.patstat import PatstatClient
patstat = PatstatClient()

In [ ]:
query="""SELECT
    EXTRACT(YEAR FROM t1.APPLN_FILING_DATE) AS filing_year,
    COUNT(DISTINCT t1.APPLN_ID) AS patent_count
FROM WYRM."300-Enriched".Patstat."2025a"."TLS201_APPLN" AS t1
JOIN WYRM."300-Enriched".Patstat."2025a"."TLS209_APPLN_IPC" AS t2
    ON t1.APPLN_ID = t2.APPLN_ID
WHERE
    t2.IPC_CLASS_SYMBOL LIKE 'H02S%'
    AND t1.APPLN_FILING_DATE >= DATE '2020-01-01'
GROUP BY
    EXTRACT(YEAR FROM t1.APPLN_FILING_DATE)
ORDER BY
    filing_year"""

res = patstat.sql_query(query, use_legacy_sql=True)
res[0:5]

This client instance is currently configured to use a test dataset with reduced number of publications (~10K).
Use PatstatClient(env='PROD') to use the complete PATSTAT dataset (>140M publications).
Use PatstatClient(env='TEST') to use the test dataset and avoid displaying this warning



QueryException: Legacy SQL dialect is currently selected. For more information, please visit the Legacy SQL documentation (https://cloud.google.com/bigquery/docs/reference/legacy-sql).

To switch to BigQuery Standard SQL, set the parameter `use_legacy_sql` to `False`.